---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
import re

In [3]:
df = pd.DataFrame(df,columns=['text'])
df

,text
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Servic...
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (N...
...,...
495,1979 Family Psych History: Family History of S...
496,therapist and friend died in ~2006 Parental/Ca...
497,2008 partial thyroidectomy\n
498,sPt describes a history of sexual abuse as a c...


In [4]:
# mm/dd/yyyy
text1 = '04/20/2009; 04/20/09; 4/20/09; 4/3/09, 4-13-89'
re.findall('(?P<Time>(?:(?P<Month>\d{1,2})[/-])(?:(?P<Day>\d{1,2})[/-])(?P<Year>\d{2,4}))',text1)

[('04/20/2009', '04', '20', '2009'),
 ('04/20/09', '04', '20', '09'),
 ('4/20/09', '4', '20', '09'),
 ('4/3/09', '4', '3', '09'),
 ('4-13-89', '4', '13', '89')]

In [5]:
# dd month yyyy
text2 = '20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009'
re.findall('(?P<Time>(?P<Day>\d{1,2}) (?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,.]? (?P<Year>\d{2,4}))',text2)

[('20 Mar 2009', '20', 'Mar', '2009'),
 ('20 March 2009', '20', 'March', '2009'),
 ('20 Mar. 2009', '20', 'Mar', '2009'),
 ('20 March, 2009', '20', 'March', '2009')]

In [6]:
# month dd yyyy
text3  = 'Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009; Mar-23-2009; Mar 24, 2009; March 25, 2009; Mar. 26, 2009; Mar 27 2009'
re.findall('(?P<Time>(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[-.]? ?(?P<Day>\d{1,2})[a-z]*(?:[-,]| ) ?(?P<Year>\d{2,4}))',text3)

[('Mar 20th, 2009', 'Mar', '20', '2009'),
 ('Mar 21st, 2009', 'Mar', '21', '2009'),
 ('Mar 22nd, 2009', 'Mar', '22', '2009'),
 ('Mar-23-2009', 'Mar', '23', '2009'),
 ('Mar 24, 2009', 'Mar', '24', '2009'),
 ('March 25, 2009', 'March', '25', '2009'),
 ('Mar. 26, 2009', 'Mar', '26', '2009'),
 ('Mar 27 2009', 'Mar', '27', '2009')]

In [7]:
# month yyyy
text4 = 'Feb 2009; Sep 2009; Oct 2010, February, 2010'
re.findall('(?P<Time>(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*),? (?P<Year>\d{2,4}))',text4)

[('Feb 2009', 'Feb', '2009'),
 ('Sep 2009', 'Sep', '2009'),
 ('Oct 2010', 'Oct', '2010'),
 ('February, 2010', 'February', '2010')]

In [8]:
# mm/yyyy
text5 = '6/2008; 12/2009'
re.findall('(?P<Time>(?P<Month>\d{1,2})/(?P<Year>\d{2,4}))',text5)

[('6/2008', '6', '2008'), ('12/2009', '12', '2009')]

In [9]:
# yyyy
text6 = '2009; 2010'
re.findall('(?P<Time>(?P<Year>\d{4}))',text6)

[('2009', '2009'), ('2010', '2010')]

In [10]:
time_full_1 = df['text'].str.extract('(?P<Time>(?:(?P<Month>\d{1,2})[/-])(?:(?P<Day>\d{1,2})[/-])(?P<Year>\d{2,4}))').dropna(how='any')
time_full_2 = df['text'].str.extract('(?P<Time>(?P<Day>\d{1,2}) (?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,.]? (?P<Year>\d{2,4}))').dropna(how='any')
time_full_3 = df['text'].str.extract('(?P<Time>(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[-.]? ?(?P<Day>\d{1,2})[a-z]*(?:[-,]| ) ?(?P<Year>\d{2,4}))').dropna(how='any')

time_df = pd.concat([time_full_1,time_full_2,time_full_3],axis=0,sort=False)

In [14]:
time_not_full_1 = df['text'].str.extract('(?P<Time>(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*),? (?P<Year>\d{2,4}))').dropna(how='any')
time_not_full_1 = time_not_full_1.loc[set(time_not_full_1.index) - set(time_df.index)]

time_not_full_2 = df['text'].str.extract('(?P<Time>(?P<Month>\d{1,2})/(?P<Year>\d{2,4}))').dropna(how='any')
time_not_full_2 = time_not_full_2.loc[set(time_not_full_2.index) - set(time_df.index)]

time_df = pd.concat([time_df,time_not_full_1,time_not_full_2],axis=0,sort=False)

In [17]:
time_not_full_3 = df['text'].str.extract('(?P<Time>(?P<Year>\d{4}))').dropna(how='any')
time_not_full_3 = time_not_full_3.loc[set(time_not_full_3.index) - set(time_df.index)]

In [18]:
time_df = pd.concat([time_df,time_not_full_3],axis=0,sort=False)
time_df.fillna(1,inplace = True)

In [19]:
time_df

,Time,Month,Day,Year
0,03/25/93,03,25,93
1,6/18/85,6,18,85
2,7/8/71,7,8,71
3,9/27/75,9,27,75
4,2/6/96,2,6,96
...,...,...,...,...
495,1979,1,1,1979
496,2006,1,1,2006
497,2008,1,1,2008
498,2005,1,1,2005


In [20]:
def month_num(x):
    if type(x) == int: return x
    elif 'Jan' in x: return 1
    elif 'Feb' in x: return 2
    elif 'Mar' in x: return 3
    elif 'Apr' in x: return 4
    elif 'May' in x: return 5
    elif 'Jun' in x: return 6
    elif 'Jul' in x: return 7
    elif 'Aug' in x: return 8
    elif 'Sep' in x: return 9
    elif 'Oct' in x: return 10
    elif 'Nov' in x: return 11
    elif 'Dec' in x: return 12
    else: return int(x)

In [21]:
time_df['Year'] = time_df['Year'].apply(lambda x: '19'+x if len(x) == 2 else x)
time_df['Month'] = time_df['Month'].apply(lambda x: month_num(x))

In [22]:
time_df_sorted = time_df.sort_values(['Year','Month','Day'],ascending=False)
time_df_sorted['Rank'] = range(500)

In [24]:
time_df_sorted['Rank'].sort_index()

0      227
1      329
2      496
3      444
4      186
      ... 
495    403
496     91
497     70
498     97
499    393
Name: Rank, Length: 500, dtype: int32